In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.journey_finder import JourneyFinder

In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt

timetable = pd.read_csv('data/timetable.csv')
footpaths = pd.read_csv('data/footpaths.csv')
stops_info = pd.read_csv('data/stops.csv')
stops_info['stop_name_id'] = stops_info['stop_name'] + " (" + stops_info['stop_id'] + ")"

journey_finder = JourneyFinder(
    timetable=timetable,
    footpaths=footpaths,
    stops_info=stops_info,
)

possible_stops = set(timetable['dep_stop']).union(set(timetable['arr_stop'])).union(set(footpaths['stop_id_a'])).union(set(footpaths['stop_id_b']))
stops_info_subset = stops_info[stops_info['stop_id'].isin(possible_stops)]

source_widget = widgets.Dropdown(
    options=sorted(list(set(stops_info_subset['stop_name_id']))),
    description='Origin Stop:',
)

destination_widget = widgets.Dropdown(
    options=sorted(list(set(stops_info_subset['stop_name_id']))),
    description='Dest. Stop:',
)

time_widget = widgets.Text(
    value='20:00:00',
    description='Arr. Time (HH:MM:SS)',
)

confidence_widget = widgets.IntSlider(
    value=90,
    min=1,
    max=100,
    step=1,
    description='Confidence:',
    orientation='horizontal',
)

button = widgets.Button(
    description='Find Journeys'
)

output = widgets.Output()
running = widgets.Label(value='')

figs = []
def update_profile_return(b):
    global figs
    
    output.clear_output()
    with output:
        running.value = 'running...'
        arrival_time = time_widget.value
        new_figs = journey_finder.find_and_plot_journeys(
            start_station_name_id=source_widget.value, 
            end_station_name_id=destination_widget.value, 
            arrival_datetime=arrival_time,
        )
        #_ = find_and_plot_journeys(timetable, footpaths, source_stop_id, destination_stop_id, arrival_time, verbose = True, num_journeys=5)
        running.value = ''
        
        # update the global figs
        figs = new_figs
        

button.on_click(update_profile_return)
output.clear_output()
display(source_widget, destination_widget, time_widget, confidence_widget, button, running, output)

Dropdown(description='Origin Stop:', options=('Bel-Air LEB (8501170)', 'Bel-Air LEB (8501170:0:1)', 'Belmont-s…

Dropdown(description='Dest. Stop:', options=('Bel-Air LEB (8501170)', 'Bel-Air LEB (8501170:0:1)', 'Belmont-su…

Text(value='20:00:00', description='Arr. Time (HH:MM:SS)')

IntSlider(value=90, description='Confidence:', min=1)

Button(description='Find Journeys', style=ButtonStyle())

Label(value='')

Output()